In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/prepped/spotify_full.csv')

In [3]:
dftotals = df.groupby(['year', 'target'])[['track']].count()
dftotals.columns = ['n_tracks']

In [4]:
dftotals

n_tracks
year target          
00s  0           2936
     1           2936
10s  0           3199
     1           3199
60s  0           4321
     1           4321
70s  0           3883
     1           3883
80s  0           3454
     1           3454
90s  0           2760
     1           2760

In [7]:
df[:2]

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target,year
0,Jealous Kind Of Fella,Garland Green,spotify:track:1dtKN6wwlolkM8XZy2y9C1,0.417,0.620,3,-7.727,1,0.0403,0.490,0.000,0.0779,0.845,185.655,173533,3,32.94975,9,1,60s
1,Initials B.B.,Serge Gainsbourg,spotify:track:5hjsmSnUefdUqzsDogisiX,0.498,0.505,3,-12.475,1,0.0337,0.018,0.107,0.1760,0.797,101.801,213613,4,48.82510,10,0,60s


In [10]:
df.groupby(['target'])[['instrumentalness']].mean()

,instrumentalness
target,
0,0.278145
1,0.030687
